# <img src="https://dl.dropboxusercontent.com/u/6117375/intermediate-notebooks/title_graphic.png" /> The Python Iteration Protocol 

### Iteration Protocols

The interpreter used to use a very simple model for iteration, which remains available today for backward compatibity reasons (though Python objects themselves no longer use it. The approximate equivalent of an old-style loop like

    for item in items:
        do_something_with(item)

would be

    intern = 0
    while True:
        try:
            item = items.__getitem__(intern)
        except IndexError:
            break
        do_something_with(item)
        intern += 1    

We normally think of iterating over the contents of a container such as a list or a string. In fact any object that responds correctly to an iteration protocol, even this now-outmoded style, can be iterated overt. Below you can see just such a class, whose items can be iterated over.

A `SequenceOf(N, obj)` object acts under iteration just like a tuple of `N` copies of the value `obj`.

In [2]:
class SequenceOf(object):
    def __init__(self, N, obj):
        self.obj = obj
        self.N = N
    def __getitem__(self, i):
        if i >= self.N:
            raise IndexError("Index exceeds defined length %d" % self.N)
        return self.obj

five_stars = SequenceOf(5, "*")
for c in five_stars:
    print(c)

three_ints = SequenceOf(3, 120)
for c in three_ints:
    print(c)

*
*
*
*
*
120
120
120


The interpreter calls its `__getitem__()` method with successively higher integers until the call raises a `StopIteration` exception. This is interpreted internally (_i.e._ the exception is trapped and handled, not appearing to the user) as the end of the sequence and the loop terminates normally.

You can see that the same objects respond to standard indexing methods pretty much in the same way as a list would (though because the implementation is in Python the traceback is a little more forthcoming than usual about what's going on. Of course it isn't very interesting, because any valid index returns the same object.

In [3]:
three_ints[1]

120

In [4]:
# XXX Turn into comment
three_ints[3]

IndexError: Index exceeds defined length 3

This older protocol does not extend to non-sequence containers such as the dict and the set, whose elements are not ordered. To a dict, numerical subscripts are just keys like any other rather than implying a position within a sequence. The set has no items to get, and therefore has no `__getitem__()` method at all!

In [5]:
"__getitem__" in dir(set())


False

If the object `iterable` _does_ have an `__iter__()` method a loop like

    for item in iterable:
        do_something_with(item)

is implemented inside the interpreter in approximately the following way.

    it = iterable.__iter__()
    while True:
        try:
            val = it.__next__()
        except StopIteration:
            break
        do_something_with(val)

With an `__iter__()` method present, the interpreter calls it to return some object which is expected to have a `__next__()` method. That method is then repeatedly calleds 

Again you observe that the exception (in the newer protocol a `StopIteration` exception must be raised to terminate iteration) never reaches the user's code, being swallowed silently as a part of the operation.

Again, once you understand how the interpreter interacts with iterable items you can implement your own. Here's a class whose instances operate similarly to `SequenceOf` objects.

In [6]:
s = set (range(10))
dir(s)

['__and__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__iand__',
 '__init__',
 '__ior__',
 '__isub__',
 '__iter__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__or__',
 '__rand__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__ror__',
 '__rsub__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__xor__',
 'add',
 'clear',
 'copy',
 'difference',
 'difference_update',
 'discard',
 'intersection',
 'intersection_update',
 'isdisjoint',
 'issubset',
 'issuperset',
 'pop',
 'remove',
 'symmetric_difference',
 'symmetric_difference_update',
 'union',
 'update']

In [7]:
class SequenceOf:
    def __init__(self, N, obj):
        self.N = N
        self.obj = obj
        self.count = 0
    def __iter__(self):
        return self
    def __next__(self):
        self.count += 1
        if self.count > self.N:
            raise StopIteration()
        return self.obj        

This object conforms to the iterator protocol because its `__iter__()` method returns an object with a `__iter__()` and a `__next__()` method. There are some implications to the fact that it returns itself htat we will look into shortly, but the fact remains that a `Sequence2` object conforms to the iterator protocol.

In [8]:
s1 = SequenceOf(5, "little strings")
for thing in s1:
    print(thing)

little strings
little strings
little strings
little strings
little strings


So that summarizes how the iteration protocol works and how you can write iterators.

There's a little more to cover, but you've got the basics.

In [9]:
from unittest.mock import MagicMock
m = MagicMock()
for x in m:
    print(m, end=" ")

In [10]:
m.mock_calls # very different from a standard interpreter - do this one on Python Anywhere? (or not at all ...)

[call.__iter__()]